In [1]:
using Random, Statistics
using FFTW

In [2]:
Nx, Ny, Nz = 8, 8, 8
R = rand(MersenneTwister(0), Nx, Ny, Nz)
R = R .- mean(R)

8×8×8 Array{Float64,3}:
[:, :, 1] =
  0.319189  -0.14263     0.0714293  …   0.447233  -0.320343    0.043884 
  0.405898   0.468758    0.363821       0.29666   -0.192313   -0.0275754
 -0.339892   0.0813535   0.463341      -0.380135  -0.308051    0.0979165
 -0.327129   0.0348312   0.263232      -0.390189   0.369123    0.286888 
 -0.225578  -0.244422   -0.0351544     -0.424904   0.150463   -0.455336 
 -0.300982   0.405588   -0.442091   …   0.272216   0.0822544  -0.0216966
 -0.462156  -0.337422   -0.151329      -0.399635   0.128237    0.160793 
 -0.436189   0.15099     0.263144       0.333617   0.230546    0.337798 

[:, :, 2] =
 -0.335351    -0.112398   -0.409666  …  -0.267658   -0.276227   -0.0693668 
  0.434399    -0.0251016   0.444426     -0.430831   -0.246833    0.482016  
  0.279268    -0.436235    0.27073      -0.39124     0.295758    0.492683  
  0.00134654  -0.02451    -0.327049      0.380707    0.211811    0.00313643
  0.117698    -0.0360611  -0.424435      0.0102109   0.0241184 

In [3]:
R2 = copy(R)
R2 .= cat(R2[:, 1:2:Ny, :], R2[:, Ny:-2:2, :]; dims=2)
R2 .= cat(R2[:, :, 1:2:Nz], R2[:, :, Nz:-2:2]; dims=3)

8×8×8 Array{Float64,3}:
[:, :, 1] =
  0.319189   0.0714293  -0.204383   …   0.447233  -0.461317  -0.14263  
  0.405898   0.363821    0.218392       0.29666   -0.236473   0.468758 
 -0.339892   0.463341   -0.384806      -0.380135  -0.437612   0.0813535
 -0.327129   0.263232    0.262612      -0.390189  -0.347821   0.0348312
 -0.225578  -0.0351544   0.297465      -0.424904   0.100839  -0.244422 
 -0.300982  -0.442091   -0.469114   …   0.272216  -0.368714   0.405588 
 -0.462156  -0.151329   -0.0197976     -0.399635   0.33366   -0.337422 
 -0.436189   0.263144    0.394741       0.333617   0.410254   0.15099  

[:, :, 2] =
 -0.438562   -0.409098    0.22255    …  -0.182645   -0.0889519   0.283615
 -0.368741    0.272639   -0.1479        -0.0604181  -0.307023    0.452669
  0.417557    0.364003   -0.0057097     -0.229005    0.0241277  -0.471865
 -0.259578    0.333314    0.301499      -0.337354    0.0807074   0.181408
 -0.439955    0.29371    -0.0640818     -0.315596   -0.101421    0.336767
 -0.0

In [14]:
@inline W(M, k) = exp(-2im*π*k/M)

ω_4Ny_ky⁺ = reshape(W.(4Ny, 0:Ny-1), 1, Ny, 1)
ω_4Ny_ky⁻ = reshape(W.(4Ny, 0:-1:-(Ny-1)), 1, Ny, 1)
ω_4Nz_kz⁺ = reshape(W.(4Nz, 0:Nz-1), 1, 1, Nz)

R3 = fft(R2, [2, 3])

r_inds = [1, collect(Ny:-1:2)...]
R3⁻ = view(R3, 1:Nx, r_inds, 1:Ny)

R4 = zeros(Nx, Ny, Nz)
@. R4 = 2 * real(ω_4Nz_kz⁺ * (ω_4Ny_ky⁺ * R3 + ω_4Ny_ky⁻ * R3⁻))

# for i in 1:Nx
#     for ky in 0:Ny-1
#         R4[i, 1, ky+1] = 4 * real(W(4Ny, ky) * W(4Nx, 0) * R3[i, 1, ky+1])
#         for kx in 1:Nx-1
#             t1 = W(4Nx,  kx) * R3[i, kx+1, ky+1]
#             t2 = W(4Nx, -kx) * R3[i, Nx-kx+1, ky+1]
#             R4[i, kx+1, ky+1] = 2 * real(W(4Ny, ky) * (t1 + t2))
#         end
#     end
# end

R4

8×8×8 Array{Float64,3}:
[:, :, 1] =
   1.12569    1.6347     -1.44021  …  -12.8604   -2.64023    -0.863094
   5.39782   14.6887      4.61822       6.31724   0.470459    0.951485
  -6.87901    0.554332    1.50472      -1.70841   0.703049   10.5593  
   0.323025   2.78682   -12.9669        1.33285  -1.36228     6.8221  
  -1.32512   -1.88894    -5.02271       1.053    -9.02383     0.750202
 -10.5928    -0.756478   -4.46764  …  -10.259     6.01476   -10.7232  
   3.48049   -0.310294    4.2778        4.85383   5.29187     2.36174 
   8.46992   -0.337076  -12.9259        7.36555   6.51841    -1.72922 

[:, :, 2] =
 -11.8192    -0.0281651  -0.63846  …   -3.9584     0.513161  -3.05912
   2.91396    7.18891     6.22586     -10.8102     6.96561   -2.29625
   0.782084  -0.813927    5.04553       3.22879    6.95248    4.11298
   1.80274   -3.8337      4.47176     -11.2742    -5.27321    7.24865
  -9.69141    1.42368     1.57058       0.155476  -6.41911    5.16523
  -5.51517   -3.47046     6.60642

In [15]:
R_DCT = FFTW.r2r(R, FFTW.REDFT10, [2, 3])

8×8×8 Array{Float64,3}:
[:, :, 1] =
   1.12569    1.6347     -1.44021  …  -12.8604   -2.64023    -0.863094
   5.39782   14.6887      4.61822       6.31724   0.470459    0.951485
  -6.87901    0.554332    1.50472      -1.70841   0.703049   10.5593  
   0.323025   2.78682   -12.9669        1.33285  -1.36228     6.8221  
  -1.32512   -1.88894    -5.02271       1.053    -9.02383     0.750202
 -10.5928    -0.756478   -4.46764  …  -10.259     6.01476   -10.7232  
   3.48049   -0.310294    4.2778        4.85383   5.29187     2.36174 
   8.46992   -0.337076  -12.9259        7.36555   6.51841    -1.72922 

[:, :, 2] =
 -11.8192    -0.0281651  -0.63846  …   -3.9584     0.513161  -3.05912
   2.91396    7.18891     6.22586     -10.8102     6.96561   -2.29625
   0.782084  -0.813927    5.04553       3.22879    6.95248    4.11298
   1.80274   -3.8337      4.47176     -11.2742    -5.27321    7.24865
  -9.69141    1.42368     1.57058       0.155476  -6.41911    5.16523
  -5.51517   -3.47046     6.60642

In [16]:
R4 ≈ R_DCT

true